## DIA 116: Crear Gráficos Estadísticos del Historial de Reportes (por Mes y Tipo)

Hoy vas a agregar gráficos 📈📊 que analizan el historial de reportes generados:

Cuántos reportes se generaron por mes

Qué porcentaje fueron PDF y CSV

✅ ¿Qué aprenderás hoy?

📊 Consultas agrupadas en SQLAlchemy

🎨 Visualizar datos con Chart.js

🔎 Análisis de actividad de usuarios

🧩 1. Ruta Flask para los datos del gráfico
python
Copiar
Editar
@app.route('/api/datos-reportes')
@login_required
def datos_reportes():
    # Agrupar por mes
    meses = db.session.query(
        db.extract('month', Reporte.fecha_generado),
        db.func.count(Reporte.id)
    ).filter_by(user_id=current_user.id).group_by(
        db.extract('month', Reporte.fecha_generado)
    ).order_by(
        db.extract('month', Reporte.fecha_generado)
    ).all()

    # Agrupar por tipo
    tipos = db.session.query(
        Reporte.tipo,
        db.func.count(Reporte.id)
    ).filter_by(user_id=current_user.id).group_by(Reporte.tipo).all()

    meses_data = {int(m): c for m, c in meses}
    tipos_data = {t: c for t, c in tipos}

    return jsonify({
        "meses": meses_data,
        "tipos": tipos_data
    })
🧩 2. Nueva ruta Flask para la página de estadísticas
python
Copiar
Editar
@app.route('/estadisticas-reportes')
@login_required
def estadisticas_reportes():
    return render_template('estadisticas_reportes.html')
🧩 3. Template estadisticas_reportes.html
html
Copiar
Editar
<!DOCTYPE html>
<html lang="es">
<head>
  <meta charset="UTF-8">
  <title>📊 Estadísticas de Reportes</title>
  <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
</head>
<body style="text-align:center;">
  <h2>📈 Estadísticas de Reportes Generados</h2>

  <div style="width:600px; margin:20px auto;">
    <canvas id="graficoMeses"></canvas>
  </div>

  <div style="width:400px; margin:20px auto;">
    <canvas id="graficoTipos"></canvas>
  </div>

  <script>
    async function cargarDatos() {
      const res = await fetch('/api/datos-reportes');
      const data = await res.json();

      const meses = ["Ene", "Feb", "Mar", "Abr", "May", "Jun", 
                     "Jul", "Ago", "Sep", "Oct", "Nov", "Dic"];

      const labelsMeses = meses;
      const valoresMeses = Array(12).fill(0);

      for (const [mes, cantidad] of Object.entries(data.meses)) {
        valoresMeses[parseInt(mes) - 1] = cantidad;
      }

      const ctxMeses = document.getElementById('graficoMeses').getContext('2d');
      new Chart(ctxMeses, {
        type: 'bar',
        data: {
          labels: labelsMeses,
          datasets: [{
            label: 'Reportes por Mes',
            data: valoresMeses,
            backgroundColor: 'steelblue'
          }]
        }
      });

      const ctxTipos = document.getElementById('graficoTipos').getContext('2d');
      new Chart(ctxTipos, {
        type: 'pie',
        data: {
          labels: Object.keys(data.tipos),
          datasets: [{
            label: 'Tipos de Reportes',
            data: Object.values(data.tipos),
            backgroundColor: ['#4CAF50', '#FFC107']
          }]
        }
      });
    }

    cargarDatos();
  </script>
</body>
</html>
